# **Crypto On-chain Data Analysis**
This notebook analyzes historical tokenomics for major cryptocurrencies (BTC, ETH, BNB, SOL) using data from CoinGecko API.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pycoingecko import CoinGeckoAPI

# Initialize CoinGecko API client
cg = CoinGeckoAPI()

# Set default template for plotly
import plotly.io as pio
pio.templates.default = "plotly_dark"


In [5]:
# Define the cryptocurrencies we want to analyze
crypto = pd.DataFrame({
    'symbol': ['BTC', 'ETH', 'BNB', 'SOL', 'XRP', 'AVAX'],
    'name': ['Bitcoin', 'Ethereum', 'Binance Coin', 'Solana', 'XRP', 'Avalanche'],
    'coingecko_id': ['bitcoin', 'ethereum', 'binancecoin', 'solana', 'ripple', 'avalanche-2']
})

print("Cryptocurrencies to analyze:")
print(crypto)

Cryptocurrencies to analyze:
  symbol          name coingecko_id
0    BTC       Bitcoin      bitcoin
1    ETH      Ethereum     ethereum
2    BNB  Binance Coin  binancecoin
3    SOL        Solana       solana
4    XRP           XRP       ripple
5   AVAX     Avalanche  avalanche-2


In [6]:
# Fetching Tokenomics Data
tokenomics_data = []

for _, row in crypto.iterrows():
    try:
        print(f"Fetching data for {row['name']}...")
        # Fetch detailed coin data from CoinGecko
        coin_data = cg.get_coin_by_id(row['coingecko_id'])
        data = {
            'symbol': row['symbol'],
            'current_price': coin_data['market_data']['current_price']['usd'],
            'market_cap': coin_data['market_data']['market_cap']['usd'],
            'total_volume': coin_data['market_data']['total_volume']['usd'],
            'circulating_supply': coin_data['market_data']['circulating_supply'],
            'total_supply': coin_data['market_data']['total_supply'],
            'max_supply': coin_data['market_data']['max_supply'],
            'fdv': coin_data['market_data'].get('fully_diluted_valuation', {}).get('usd'),
        }
        tokenomics_data.append(data)
        
    except Exception as e:
        print(f"Error fetching data for {row['symbol']}: {e}")

# Convert to DataFrame
tokenomics_df = pd.DataFrame(tokenomics_data)

Fetching data for Bitcoin...
Fetching data for Ethereum...
Fetching data for Binance Coin...
Fetching data for Solana...
Fetching data for XRP...
Fetching data for Avalanche...


In [21]:
# 1. Market Cap Comparison (Bar Chart)
fig1 = px.bar(
    tokenomics_df,
    x='symbol',
    y='market_cap',
    title='Market Capitalization by Cryptocurrency',
    labels={'market_cap': 'Market Cap (USD)', 'symbol': 'Cryptocurrency'},
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
)

# Customize the layout
fig1.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig1.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [30]:
# 2. Supply Metrics Comparison (Grouped Bar Chart)
supply_data = pd.melt(
    tokenomics_df,
    id_vars=['symbol'],
    value_vars=['circulating_supply', 'total_supply', 'max_supply'],
    var_name='supply_type',
    value_name='supply',
)
fig2 = px.bar(
    supply_data,
    x='symbol',
    y='supply',
    color='supply_type',
    title='Supply Metrics by Cryptocurrency',
    barmode='group',
    labels={'supply': 'Supply Amount', 'symbol': 'Cryptocurrency'}
)

# Customize the layout
fig2.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

fig2.show()

In [43]:
# 3. Market Cap vs Volume (Scatter Plot)
fig3 = px.scatter(
    tokenomics_df,
    x='market_cap',
    y='total_volume',
    text='symbol',
    title='Market Cap vs Trading Volume',
    labels={
        'market_cap': 'Market Cap (USD)',
        'total_volume': 'Trading Volume (USD)'
    },
    size='market_cap',
    size_max=150,
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
    log_x=True,
    log_y=True
)

# Customize the layout
fig3.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig3.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [42]:
# 4. Price vs Supply (Bubble Chart)
fig4 = px.scatter(
    tokenomics_df,
    x='circulating_supply',
    y='current_price',
    size='market_cap',
    size_max=150,
    text='symbol',
    title='Price vs Circulating Supply (Bubble size = Market Cap)',
    labels={
        'circulating_supply': 'Circulating Supply',
        'current_price': 'Price (USD)',
    },
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
    log_x=True,
    log_y=True
)

# Customize the layout
fig4.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig4.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [35]:
# 5. Market Dominance (Pie Chart)
fig5 = px.pie(
    tokenomics_df,
    values='market_cap',
    names='symbol',
    title='Market Cap Distribution',
    color='symbol',
    color_discrete_map={
        'BTC': '#F7931A',  # Bitcoin Orange
        'ETH': '#636BFF',  # Ethereum Blue
        'BNB': '#F3BA2F',  # Binance Yellow
        'SOL': '#CF62E7',   # Solana Pink
        'XRP': '#4B8BBE',  # XRP Blue
        'AVAX': '#E84142'  # Avalanche Red
    },
)

# Customize the layout
fig5.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
)

# Show the plot
fig5.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

In [44]:
# Add new cell for time series data
from datetime import datetime, timedelta
import time

# Get data for last 365 days
days = 365
price_history = {}

for _, row in crypto.iterrows():
    try:
        print(f"Fetching historical data for {row['name']}...")
        # Rate limiting to avoid API issues
        time.sleep(1)
        
        history = cg.get_coin_market_chart_by_id(
            id=row['coingecko_id'],
            vs_currency='usd',
            days=days,
            interval='daily'
        )
        
        # Extract timestamp and price
        prices_df = pd.DataFrame(
            history['prices'], 
            columns=['timestamp', row['symbol']]
        )
        prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'], unit='ms')
        
        if len(price_history) == 0:
            price_history = prices_df
        else:
            price_history = price_history.merge(
                prices_df, 
                on='timestamp', 
                how='outer'
            )
            
    except Exception as e:
        print(f"Error fetching historical data for {row['symbol']}: {e}")

# Create price history visualization
fig6 = px.line(
    price_history, 
    x='timestamp',
    y=crypto['symbol'].tolist(),
    title='Price History (365 Days)',
    labels={'timestamp': 'Date', 'value': 'Price (USD)'},
    color_discrete_map={
        'BTC': '#F7931A',
        'ETH': '#636BFF',
        'BNB': '#F3BA2F',
        'SOL': '#CF62E7',
        'XRP': '#4B8BBE',
        'AVAX': '#E84142'
    }
)

# Customize the layout
fig6.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    height=800/1.25,
    width=1200/1.25,
    yaxis_type="log"  # Log scale for better price comparison
)

# Show the plot
fig6.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

Fetching historical data for Bitcoin...
Fetching historical data for Ethereum...
Fetching historical data for Binance Coin...
Fetching historical data for Solana...
Fetching historical data for XRP...
Fetching historical data for Avalanche...
